# 인공지능 작곡가

단어를 입력으로 주면 다음 단어를 예측해 문장을 만들어갑니다.

데이터셋 출처 : 
https://www.kaggle.com/paultimothymooney/poetry

**라이브러리 불러오기 및 난수 고정**

가중치값 초기화, shuffle 등의 기능에서 생성된 시드값에 따라 결과가 바뀌곤 합니다. <br/>`tf.random.set_seed()`를 통해 코드를 재생산 할 수 있게합니다.

In [1]:
# 필요 라이브러리 불러오기
import os, glob, re, random
import tensorflow as tf
from sklearn.model_selection import train_test_split

# 결과 복원을 위해 난수 생성 고정

tf.random.set_seed(7777)

**파일 읽기**

다운받은 가사가 들어있는 폴더를 지정하여 안에있는 파일들을 모두 읽습니다.<br/>
이를 한 줄씩 읽어 `raw_corpus` 변수에 저장합니다.<br/>
이 과정에서 원치않는 숨김폴더들이 있는 경우 예외처리가 필요할 수 있습니다.

In [2]:
file_path = '/content/drive/MyDrive/aiffel/ex4/lyrics/*'

file_list = glob.glob(file_path)
raw_corpus = []

for file_ in file_list:
    with open(file_, "r") as f:
        line = f.read().splitlines()
        raw_corpus.extend(line)

print("가사 파일 수 : ", len(file_list))
print("가사의 줄 수 : ", len(raw_corpus))

가사 파일 수 :  49
가사의 줄 수 :  187088


**데이터 전처리**

한줄 씩 읽은 가사를 학습용 데이터로 변환합니다. 문장의 시작과 끝을 알리는 <start>와 <end> 를 기본으로하고 특수문자들을 처리합니다. 또한 추가적으로 너무 짧은 문장과 너무 긴 문장을 제거했습니다. 마지막으로 문장이 `'('` 로 시작하는 경우 `'['`로 시작하는 경우는 추임새와 후렴구가 포함되는 경우가 많았습니다. 대게의 경우 완성된 문장으로 보기 어렵다 판단하여 제거하였습니다. 

In [3]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # ?.!,¿ 를 만나면 앞뒤를 공백으로 구분합니다.
    sentence = re.sub(r'[" "]+', " ", sentence) # 공백이 여러개 있는 부분을 한개로 줄여요
    sentence = re.sub(r"[^a-zA-Z'?.!,¿]+", " ", sentence) # a-zA-Z'?.!,¿ 이에 해당되지 않는 또 다른 특수문자를 공백으로 바꿉니다.
    sentence = sentence.strip() # 앞뒤 쓸데없는 공백을 제거해요.
    sentence = '<start> ' + sentence + ' <end>' # 모델 학습을 위해 문장의 시작을 알리는 <start> 토큰와 끝을 알리는 <end> 토큰을 추가합니다.
    return sentence

corpus = [] # 전처리 되지 않은 raw_corpus를 읽어서 전처리한 corpus를 저장합니다.

for sentence in raw_corpus:
    if len(sentence) == 0: continue # 줄 단위로 raw_corpus에 저장했으니 단순 줄바꿈이 있을 수 있습니다. 제거해줍니다.
    if sentence[-1] == ":": continue # 그룹의 노래일 경우 가수마다 파트가 나눠져있습니다. "<가수이름> : " 제거합니다.
    if sentence.startswith('('): continue # 추임새 또는 후렴구가 포함되어 있는 경우가 많았습니다.
    if sentence.startswith('['): continue # 위와 같음
    if len(set(sentence.split())) < 3: continue # 같은 말 반복(Yeah, Yeah, Yeah, Yeah)되거나 I love, 같은 짧은 문장 제거
    if len(sentence.split()) > 13: continue # 너무 긴 문장을 제거합니다.

    # 위의 조건에 해당되지 않는 경우만 preprocess를 합니다. 굳이 전처리를 할 필요도 없이 제거해버린거죠.
        
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
    
random.sample(corpus, 5) # 임의의 5래를 출력해봅니다~

['<start> here are some who like to run . <end>',
 '<start> black timbs all on your couch again <end>',
 "<start> i'm not gonna stand here and wait <end>",
 '<start> buying food every once in a while <end>',
 "<start> i'm like ooh the girl deserve it , man , nobody so perfect <end>"]

**토큰화**

앞서 구한 corpus를 통해 텍스트를 여러개의 토큰으로 만듭니다.<br/>
전체 사전을 만든다고 이해하면 됩니다. 사전에 담을 단어의 수는 아래와 같이 10000개입니다. 크면 클수록 더 다양한 문장을 만들 수 있습니다. <br/>
만든 사전을 기반으로 corpus의 각 문장을 텐서의 형태로 변환합니다. 컴퓨터는 텍스트가 아니라 Tensor의 입력을 처리할 수 있기 때문입니다. 만든 사전에는 단어에 대한 값이 할당되어 있습니다. 이를 통해 `sequences_to_texts` 혹은 `texts_to_sequences`를 수행할 수 있습니다.

In [4]:
def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=10000, 
        filters=' ',
        oov_token="<unk>",
    )
    tokenizer.fit_on_texts(corpus)
    tensor = tokenizer.texts_to_sequences(corpus)   
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')  
    
    return tensor, tokenizer
tensor, tokenizer = tokenize(corpus)

**X, Y 분리**

각 문장을 텐서로 만들었습니다. RNN 학습은 앞서 나온 단어를 입력으로 뒤에 나올 단어를 출력으로 합니다. 시작(`<start>`)부터 마지막(`<end>`)전까지를 입력 데이터로, 시작(`<start>`+1)부터 마지막(`<end>`)까지를 출력 데이터로 설정합니다.

In [5]:
src_input = tensor[:, :-1]
tgt_input = tensor[:, 1:]

**Train, Val** 분리

앞서 구한 `X`, `Y`를 train을 위한 데이터, val을 위한 데이터로 쪼갭니다. 여기서 특이한점은 test을 위한 데이터가 없는데, RNN 학습에서는 특별히 test 데아터로 결과를 평가할 수 없기 때문에 나누지 않았다. text generate를 통해 적절한 문장이 생성되었는지 확인한다.

In [6]:
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size=0.2, random_state=7777)

데이터들의 형태를 확인한다.

In [7]:
print("encode train:", enc_train.shape)
print("decode Train:", dec_train.shape)
print("encode val:", enc_val.shape)
print("decode val:", enc_val.shape)

encode train: (120839, 32)
decode Train: (120839, 32)
encode val: (30210, 32)
decode val: (30210, 32)


`TextGenerator` 모델의 구조를 정의한다.

- `Embedding`
- `LSTM`
- `Dropout`
- `LSTM`
- `Dense`

In [8]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size1, hidden_size2):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size1, return_sequences=True)
        self.dropout = tf.keras.layers.Dropout(0.2)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size2, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.dropout(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out

데이터의 전처리가 가장 중요하고 다음으로 모델의 구조가 중요하다. 그 다음으로 중요한 것은 모델을 구성하는 **하이퍼 파라미터**다. <br/>

간단하게 `LSTM`레이어의 hidden layer수를 조정하여 모델의 결과를 지켜보고자 하였다. 

In [9]:
from itertools import product

embedding_size = 256
VOCAB_SIZE = tokenizer.num_words + 1

best_val_loss = 100
best_hidden1 = 0
best_hidden2 = 0

hidden_sizes1 = [1024, 2048, 4096]
hidden_sizes2 = [1024, 2048, 4096]

for hidden_size1, hidden_size2 in product(hidden_sizes1, hidden_sizes2):
    model = TextGenerator(VOCAB_SIZE, embedding_size, hidden_size1, hidden_size2)

    optimizer = tf.keras.optimizers.Adam()
    loss = tf.keras.losses.SparseCategoricalCrossentropy(
        from_logits=True,
        reduction='none'
    )

    model.compile(loss=loss, optimizer=optimizer)

    history = model.fit(enc_train, dec_train, validation_data=(enc_val, dec_val), batch_size=256, epochs=10)
    val_loss = history.history['val_loss'][-1]
    if best_val_loss > val_loss:
        best_val_loss = val_loss
        best_hidden1 = hidden_size1
        best_hidden2 = hidden_size2
    print("\n"*5)
    print(f"hidden_size1 : {hidden_size1}, hidden_size2: {hidden_size2}, loss: {val_loss}")
    print("-"*30)
    print("\n"*5)

위의 코드를 통해 얻은 결과는 다음과 같다.

- [hidden1, hidden2] -> val_loss
- [1024, 1024] -> 1.16
- [1024, 2048] -> 1.04
- [1024, 4096] -> 1.07
- [2048, 1024] -> 1.14
- [2048, 2048] -> 1.16
- [2048, 4096] -> 1.04
- [4096, 1024] -> 1.09
- [4096, 2048] -> 1.08
- [4096, 4096] -> 1.12


*지면의 길이가 너무 길어 결과를 추가하지 않았다.*

**가장 좋은 결과를 낸 모델로 다시 모델 학습**

위의 결과에서 볼 수 있듯이 [1024, 2048], [2048, 4096]이 좋은 결과를 냈지만 굳이 더 큰 모델을 사용할 필요는 없었다. 따라서 [1024, 2048]을 사용해서 다시 학습한다.

In [10]:
VOCAB_SIZE = tokenizer.num_words + 1
embedding_size = 256

model = TextGenerator(VOCAB_SIZE, embedding_size, 1024, 2048)

optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)

history = model.fit(enc_train, dec_train, validation_data=(enc_val, dec_val), batch_size=256, epochs=10)

Epoch 1/10
473/473 [==============================] - 200s 392ms/step - loss: 1.6055 - val_loss: 1.4285
Epoch 2/10
473/473 [==============================] - 184s 390ms/step - loss: 1.3939 - val_loss: 1.3327
Epoch 3/10
473/473 [==============================] - 185s 390ms/step - loss: 1.2659 - val_loss: 1.2596
Epoch 4/10
473/473 [==============================] - 184s 390ms/step - loss: 1.1629 - val_loss: 1.1980
Epoch 5/10
473/473 [==============================] - 184s 390ms/step - loss: 1.0583 - val_loss: 1.1481
Epoch 6/10
473/473 [==============================] - 185s 390ms/step - loss: 0.9570 - val_loss: 1.1084
Epoch 7/10
473/473 [==============================] - 185s 390ms/step - loss: 0.8604 - val_loss: 1.0773
Epoch 8/10
473/473 [==============================] - 185s 390ms/step - loss: 0.7734 - val_loss: 1.0571
Epoch 9/10
473/473 [==============================] - 185s 390ms/step - loss: 0.6984 - val_loss: 1.0454
Epoch 10/10
473/473 [==============================] - 185s 390m

**텍스트를 생성해주는 함수 생성**

model, tokenizer, init_sentence, maxlen을 인자로 받아 maxlen에 도달하거나 `<end>` 토큰을 생성할 때까지 문장을 생성합니다.

모델에 문장를 토큰화한 Tensor를 입력으로 주면 다음 단어를 예측해서 결과를 줍니다. 단어는 입력 문장과 합쳐집니다. 이를 앞선 조건이 맞을 때까지 반복합니다.

In [11]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=100):
    
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    while True:
        predict = model(test_tensor) 
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

In [21]:
generate_text(model, tokenizer, init_sentence="<start> i love")

'<start> i love you so much , so o o o <end> '

In [13]:
generate_text(model, tokenizer, init_sentence="<start> what")

'<start> what you want be what you want <end> '

In [14]:
generate_text(model, tokenizer, init_sentence="<start> let")

'<start> let me take a ride <end> '

In [15]:
generate_text(model, tokenizer, init_sentence="<start> give")

'<start> give me a spanish sweet hoe <end> '

In [16]:
generate_text(model, tokenizer, init_sentence="<start> hello")

'<start> hello hello hello how low <end> '

In [17]:
generate_text(model, tokenizer, init_sentence="<start> kiss")

'<start> kiss me , kiss me <end> '

In [18]:
generate_text(model, tokenizer, init_sentence="<start> happy")

'<start> happy , yeah , yeah <end> '

**의문점들**

- generate_text 함수는 최대 100개의 단어를 가질 수 있도록 만들었는데, 왜 이렇게 짧은 문장으로 끝이난걸까? 입력의 최대 길이를 15로 했기 때문인가? 그렇다면 소설을 쓰는 인공지능은 어떠한 데이터로 학습이 된걸까?
- 모델에 입력데이터로 tensor의 (시작: 끝-1), 출력데이터로 tensor (시작+1: 끝) 이렇게 모델에 주었는데, 어떠한 식으로 학습이 되는걸까? 입력한 대로라면 처음부터 끝까지의 입력이 들어가고 마지막 단어만을 예측한다. 그렇다면 마지막 단어만 예측하는 잘못된 입력이라고 생각한다.
- batch_size에 따라 학습의 결과가 정말 많이 차이가 났다. GPU의 RAM이 감당할 수 있는 최대의 90% 정도로 BATCH_SIZE를 설정해주면 적절하다고 알고 있었는데 생각과 다른 학습이였다.

### 결론

가장 큰 걸림돌은 시간이였다. 학습을 돌리고 기다리는 시간, 이 시간을 줄일 수 없다면 차라리 길게해서 다른 일에 집중할 수 있도록 코드를 구성해야겠다는 생각이 들었다. 하나의 모델에서 hyperparameter tuning을 값을 변경하며 학습을 하는 과정은 간단하다. 하지만 더 큰 문제는 모델의 구조가 적절한지라고 생각한다. RNN, GRU, LSTM, Bi-LSTM 등 다양한 기법들이 있지만 이를 어떻게 조합해야 하는지 아는것은 더욱 중요한 과제이다. 각각의 레이어들이 어떠한 방식으로 동작하는 지 데이터에 따른 어떠한 레이어가 적절한지에 대해 더 공부하자.